In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!tar xzvf /content/gdrive/MyDrive/darknet/cudnn/cudnn-10.1-linux-x64-v8.0.4.30.tgz -C /usr/local/

cuda/include/cudnn.h
cuda/include/cudnn_adv_infer.h
cuda/include/cudnn_adv_train.h
cuda/include/cudnn_backend.h
cuda/include/cudnn_cnn_infer.h
cuda/include/cudnn_cnn_train.h
cuda/include/cudnn_ops_infer.h
cuda/include/cudnn_ops_train.h
cuda/include/cudnn_version.h
cuda/NVIDIA_SLA_cuDNN_Support.txt
cuda/lib64/libcudnn.so
cuda/lib64/libcudnn.so.8
cuda/lib64/libcudnn.so.8.0.4
cuda/lib64/libcudnn_adv_infer.so
cuda/lib64/libcudnn_adv_infer.so.8
cuda/lib64/libcudnn_adv_infer.so.8.0.4
cuda/lib64/libcudnn_adv_train.so
cuda/lib64/libcudnn_adv_train.so.8
cuda/lib64/libcudnn_adv_train.so.8.0.4
cuda/lib64/libcudnn_cnn_infer.so
cuda/lib64/libcudnn_cnn_infer.so.8
cuda/lib64/libcudnn_cnn_infer.so.8.0.4
cuda/lib64/libcudnn_cnn_train.so
cuda/lib64/libcudnn_cnn_train.so.8
cuda/lib64/libcudnn_cnn_train.so.8.0.4
cuda/lib64/libcudnn_ops_infer.so
cuda/lib64/libcudnn_ops_infer.so.8
cuda/lib64/libcudnn_ops_infer.so.8.0.4
cuda/lib64/libcudnn_ops_train.so
cuda/lib64/libcudnn_ops_train.so.8
cuda/lib64/libcudnn_o

In [ ]:
!cat /usr/local/cuda/include/cudnn_version.h | grep CUDNN_MAJOR -A 2

#define CUDNN_MAJOR 8
#define CUDNN_MINOR 0
#define CUDNN_PATCHLEVEL 4
--
#define CUDNN_VERSION (CUDNN_MAJOR * 1000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

#endif /* CUDNN_VERSION_H */


In [ ]:
%cd /content/gdrive/MyDrive/darknet/bin/darknet

/content/gdrive/MyDrive/darknet/bin/darknet


In [ ]:
!chmod +x ./darknet

In [ ]:
%ls

1st.mov                darknet*                  re.txt
50to300_flip/          fastandfurious_short.mov  train/
Amber_contrast/        loss_chart/               train_2/
bad_label.list         makefile_for_test.py      train_2.1_1000_frames_tiny/
bad.list               makefile_for_train.py     train_2.1_300_frames/
chart.png              make_path_txt.py          train_2.1_600_frames_tuned_v4/
chart_yolov3.png       predicted_img/            train_2.1_600_frames_v3/
chart_yolov3-tiny.png  predictions.jpg           train_2.1_900_frames_v3/
chart_yolov4.png       __pycache__/              train_2.1_rb/


In [1]:
# Train
!LD_LIBRARY_PATH=/usr/local/cuda/lib64 ./darknet detector train train_2.1_600_frames_tuned_v4/r1mini.data train_2.1_600_frames_tuned_v4/yolov4.cfg -dont_show

In [2]:
# mAP 확인
!LD_LIBRARY_PATH=/usr/local/cuda/lib64 ./darknet detector map train_2.1_600_frames_tuned_v4/r1mini.data train_2.1_600_frames_tuned_v4/yolov4.cfg train_2.1_600_frames_tuned_v4/backup/yolov4_6000.weights

In [3]:
# video train 확인
!LD_LIBRARY_PATH=/usr/local/cuda/lib64 ./darknet detector demo train_2.1_600_frames_tuned_v4/r1mini.data train_2.1_600_frames_tuned_v4/yolov4.cfg train_2.1_600_frames_tuned_v4/backup/yolov4_4000.weights ./fastandfurious_short.mov -i 0 -out_filename 600fr_v4.mov -dont_show

In [4]:
# ROI
!LD_LIBRARY_PATH=/usr/local/cuda/lib64 ./darknet detector test train_2/r1mini.data train_2/yolov3.cfg train_2/backup/yolov3_last.weights -ext_output train_2/r1mini/test.jpg

In [ ]:
# 폴더에 있는 모든 img를 predict하기

# 변수 선언

#1. predict할 파일 들어있는 폴더 (txt파일에 입력될 path기 때문에 절대경로 추천, 마지막에 꼭 '/' 넣어줘야함.)
image_directory = '/content/gdrive/MyDrive/darknet/bin/darknet/train_2.1_600_frames_tuned_v4/final_test/'
#2. 확장자
extension = "*.jpg"
#3. txt파일 저장경로 및 파일 이름
save_at = './re.txt'
#4. predicted_pics 저장 경로+ 파일 이름 ex) predicted_img0.jpg,predicted_img1, ~ predicted_img10 로 저장할거면 아래처럼
predicted_pics_path ='/content/gdrive/MyDrive/darknet/bin/darknet/predicted_img/600_frames_v4_wt2000/'

# ***************************************************************************************************
# 아래는 걍 이것저것 편하게 넣어보려고 함수화한 것, 불필요하면 해당 자리에 그냥 사용하시는 커맨드 + $img_path 입력하면 됨 (.py로 선언 불가)
# ex) !./darknet detector test train/r1mini.data train/yolov3.cfg train/backup/yolov3_final.weights $img_path
#5. .data 파일 경로
data_path = '/content/gdrive/MyDrive/darknet/bin/darknet/train_2.1_600_frames_tuned_v4/r1mini.data'
#6. config 파일 경로
cfg_path = '/content/gdrive/MyDrive/darknet/bin/darknet/train_2.1_600_frames_tuned_v4/yolov4.cfg'
#7. weights 파일 경로
weights_path = '/content/gdrive/MyDrive/darknet/bin/darknet/train_2.1_600_frames_tuned_v4/backup/yolov4_2000.weights'

#command 함수
def darknet_predict_img(data_path,cfg_path,weights_path,img_path):
    !LD_LIBRARY_PATH=/usr/local/cuda/lib64 ./darknet detector test $data_path $cfg_path $weights_path $img_path


import glob
import PIL
import PIL.Image as Image
import make_path_txt
import os



d=0
with open(make_path_txt.make_data_path(image_directory,extension, save_at),'r') as fobj:
        image_List = [[num for num in line.split()] for line in fobj]
        for images in image_List:
            # print(images[0])
            img_path=images[0]
            darknet_predict_img(data_path,cfg_path,weights_path,img_path)
            #다크넷 경로 + prdictions.jpg
            predicted_image = Image.open(str(os.getcwd())+'/'+'predictions'+extension[1:])
            #predict 결과 저장할 경로+ 파일이름
            output = predicted_pics_path + '{}{}'.format(d,extension[1:])
            predicted_image.save(output) 
            d+=1

Streaming output truncated to the last 5000 lines.
Cannot load image data/labels/61_7.png
Cannot load image data/labels/62_7.png
Cannot load image data/labels/63_7.png
Cannot load image data/labels/64_7.png
Cannot load image data/labels/65_7.png
Cannot load image data/labels/66_7.png
Cannot load image data/labels/67_7.png
Cannot load image data/labels/68_7.png
Cannot load image data/labels/69_7.png
Cannot load image data/labels/70_7.png
Cannot load image data/labels/71_7.png
Cannot load image data/labels/72_7.png
Cannot load image data/labels/73_7.png
Cannot load image data/labels/74_7.png
Cannot load image data/labels/75_7.png
Cannot load image data/labels/76_7.png
Cannot load image data/labels/77_7.png
Cannot load image data/labels/78_7.png
Cannot load image data/labels/79_7.png
Cannot load image data/labels/80_7.png
Cannot load image data/labels/81_7.png
Cannot load image data/labels/82_7.png
Cannot load image data/labels/83_7.png
Cannot load image data/labels/84_7.png
Cannot load i

In [ ]:
# Train 중단뒤 다시 연결하기
!LD_LIBRARY_PATH=/usr/local/cuda/lib64 ./darknet detector train train_3/r1mini.data train_3/yolov3.cfg train_3/backup/yolov3_2000.weights -dont_show